In [48]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tensorflow import keras
import torch
from keras.preprocessing.image import ImageDataGenerator
from torch.utils.data import Dataset
from torchvision import transforms,datasets 
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import cv2
from sklearn.preprocessing import MultiLabelBinarizer
import random
from tensorflow.keras.applications import ResNet50
import csv
from PIL import Image
import torch.optim as optim
from transformers import BertTokenizer, TFBertModel, BertModel
from tensorflow.keras.optimizers import Adam
import torch.nn as nn

In [59]:
# Define the path to the CSV file and image directory
csv_file = 'train_cap_label.csv'
image_dir = 'path/to/image/directory'

In [95]:
# Load the data from the CSV file
data = []
with open(csv_file, 'r') as file:
    csv_reader = csv.reader(file)
    for row in csv_reader:
        if len(row) == 2:
            labels = [int(label) - 1 for label in row[0].split() if label.strip()]  # Subtract 1 from labels
            caption = row[1]
            if labels:  # Only append if there are labels
                data.append((labels, caption))

# Split the data into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.5, random_state=42)

In [96]:
# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [97]:
# Tokenize the captions
train_captions = [caption for _, caption in train_data]
train_labels = [labels for labels, _ in train_data]
test_captions = [caption for _, caption in test_data]
test_labels = [labels for labels, _ in test_data]

train_encodings = tokenizer(train_captions, truncation=True, padding=True)
test_encodings = tokenizer(test_captions, truncation=True, padding=True)

In [98]:
# Convert labels to one-hot encoded vectors
num_classes = 18  # Number of classes
train_labels_onehot = tf.keras.utils.to_categorical([tf.reduce_max(labels) for labels in train_labels], num_classes=num_classes)
test_labels_onehot = tf.keras.utils.to_categorical([tf.reduce_max(labels) for labels in test_labels], num_classes=num_classes)


In [99]:
# Create the BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [103]:
# Define the model architecture
input_ids = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name='attention_mask')
bert_output = bert_model(input_ids, attention_mask=attention_mask)
pooled_output = bert_output.pooler_output
hidden_layer = tf.keras.layers.Dense(32, activation='relu')(pooled_output)
output_layer = tf.keras.layers.Dense(num_classes, activation='sigmoid')(hidden_layer)
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output_layer)

In [104]:
# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00002),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Train the model
batch_size = 256
epochs = 3
model.fit(
    {'input_ids': np.array(train_encodings['input_ids']),
     'attention_mask': np.array(train_encodings['attention_mask'])},
    train_labels_onehot,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(
        {'input_ids': np.array(test_encodings['input_ids']),
         'attention_mask': np.array(test_encodings['attention_mask'])},
        test_labels_onehot
    )
)

Epoch 1/3
59/59 [==============================] - 3303s 56s/step - loss: 0.4095 - accuracy: 0.2419 - val_loss: 0.2517 - val_accuracy: 0.4675
Epoch 2/3
31/59 [==============>...............] - ETA: 19:26 - loss: 0.2362 - accuracy: 0.4744

In [ ]:
# Generate predictions on the test set
test_predictions = model.predict(
    {'input_ids': np.array(test_encodings['input_ids']),
     'attention_mask': np.array(test_encodings['attention_mask'])}
)

In [ ]:
# Convert predictions to binary labels
threshold = 0.5
binary_predictions = np.where(test_predictions >= threshold, 1, 0)

# Calculate the F1-score
f1 = f1_score(test_labels, binary_predictions, average='micro')
print(f'F1-score: {f1:.4f}')